# Conv(Normal, Dilated, Depthwise separable, Spatially separable) BatchNorm Relu

In [0]:
import tensorflow as tf

In [0]:
class BatchNorm(tf.keras.Model):
  
  def __init__(self, momentum=0.9, epsilon=1e-5):
    
    super().__init__()
    
    self.momentum = momentum
    
    self.epsilon = epsilon
    
    self.bn = tf.keras.layers.BatchNormalization(momentum=self.momentum, epsilon=self.epsilon)
  
  def call(self, inputs):
    
    return self.bn(inputs)

In [0]:
class spatially_separable_conv(tf.keras.Model):
  
  def __init__(self, filters=32, kernel_size=(3,3), strides=(1,1), padding="same" , dilation_rate=(1,1), kernel_regularizer = None, 
               kernel_initializer="glorot_uniform"):
    
    super().__init__()
    
    k_size1 = (kernel_size[0], 1)
    
    k_size2 = (1, kernel_size[1])
    
    self.step1 = tf.keras.layers.Conv2D(filters=filters, kernel_size=k_size1, strides=strides, padding=padding, dilation_rate=dilation_rate,
                                       kernel_regularizer=kernel_regularizer, kernel_initializer=kernel_initializer, use_bias=False)
    
    self.step2 = tf.keras.layers.Conv2D(filters=filters, kernel_size=k_size1, strides=strides, padding=padding, dilation_rate=dilation_rate,
                                       kernel_regularizer=kernel_regularizer, kernel_initializer=kernel_initializer, use_bias=False)
    
  def call(self, inputs):
      
    return self.step2(self.step1(inputs))

In [0]:
class ConvBnRl(tf.keras.Model):
  
  def __init__(self, filters=32, kernel_size=(3,3), strides=(1,1), padding="same" , dilation_rate=(1,1), kernel_regularizer = None, 
               kernel_initializer="glorot_uniform", conv_flag=True, bnflag=True,  relu=True, depthwise_separable=False, spatial_separable=False):
    
    super().__init__()
    
    self.relu = relu
    
    self.bn_flag = bnflag
    
    self.conv_flag = conv_flag
    
    if depthwise_separable:
      
      self.conv = tf.keras.layers.SeparableConv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding, dilation_rate=dilation_rate,
                                       kernel_regularizer=kernel_regularizer, kernel_initializer=kernel_initializer, use_bias=False)
    elif spatial_separable:
      
      self.conv = spatially_separable_conv(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding, dilation_rate=dilation_rate, 
                                           kernel_regularizer = kernel_regularizer, kernel_initializer=kernel_initializer)
      
    else:
      
      self.conv = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding, dilation_rate=dilation_rate,
                                       kernel_regularizer=kernel_regularizer, kernel_initializer=kernel_initializer, use_bias=False)
    
    
    
    self.bn = BatchNorm(momentum=0.9, epsilon=1e-5)

  def call(self, inputs):
    
    if self.conv_flag:

      if self.relu:

        if self.bn_flag:
          return tf.nn.relu(self.bn(self.conv(inputs)))

        else:
          return tf.nn.relu(self.conv(inputs))
      else:

        if self.bn_flag:
          return self.bn(self.conv(inputs))

        else:
          return self.conv(inputs)
        
    else:
      
      if self.relu:

        if self.bn_flag:
          
          return tf.nn.relu(self.bn(inputs))

        else:
          
          return tf.nn.relu(inputs)
      else:

        if self.bn_flag:
          
          return self.bn(inputs)

        else:
          
          """if conv, bn, rl flags are False, then just return conv"""
          
          return self.conv(inputs)

In [0]:
import numpy as np

In [7]:
x = ConvBnRl(filters=32, kernel_size=(1,1), strides=(1,1), padding="valid" , dilation_rate=(1,1), kernel_regularizer = None, 
               kernel_initializer="glorot_uniform", conv_flag=True, bnflag=True,  relu=True, depthwise_separable=True)

x(np.random.normal(size=(32,32,3)).reshape(1,32,32,3).astype(np.float16))

W0830 15:53:35.675567 140616057554816 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


<tf.Tensor 'conv_bn_rl/Relu:0' shape=(1, 32, 32, 32) dtype=float16>

# ResNet

In [0]:
class ResBlk(tf.keras.Model):
  
  def __init__(self, 
               
               cbr = ConvBnRl(filters=32, kernel_size=(3,3), strides=(1,1), padding="same" , conv_flag=True, bnflag=True, relu=True),
               
               cbr_res1 = ConvBnRl(filters=32, kernel_size=(3,3), strides=(1,1), padding="same" , conv_flag=True, bnflag=True, relu=True),
               
               cbr_res2 = ConvBnRl(filters=32, kernel_size=(3,3), strides=(1,1), padding="same" , conv_flag=True, bnflag=True, relu=True),
               
               pool=tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2,2), padding='same'), 
               
               res=True ):
    
    super().__init__()
    
    self.conv_bn = cbr
    
    self.pool = pool
    
    self.res = res
    
    if self.res:
      
      self.res1 = cbr_res1
      
      self.res2 = cbr_res2

  def call(self, inputs):
    
    h = self.pool(self.conv_bn(inputs))
    
    if self.res:
      
      h = h + self.res2(self.res1(h))
      
    return h

In [9]:
a = ResBlk()

a(x(np.random.normal(size=(32,32,3)).reshape(1,32,32,3).astype(np.float16)))

<tf.Tensor 'res_blk/add:0' shape=(1, 16, 16, 32) dtype=float16>

# ResNext

In [0]:
class ResNeXtBlk(tf.keras.Model):
  
    def __init__(self, 
                 
                 layer_num, 
                 
                 filters=32, 
                 
                 pool=tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=None, padding='same'), 
                 
                 kernel_regularizer=None,
                 
                 kernel_initializer="glorot_uniform",
                 
                 res_block=1, cardinality=8):
      
      super().__init__()
      
      self.filters = filters
      
      self.layer_num = layer_num
      
      self.res_block=res_block
      
      self.kernel_regularizer = kernel_regularizer
      
      self.kernel_initializer = kernel_initializer
      
      self.conv_bn_1x1 = ConvBnRl(filters=self.filters, kernel_size=(1,1), strides=(1,1), padding="same" , dilation_rate=(1,1), 
                                  kernel_regularizer = self.kernel_regularizer, kernel_initializer=self.kernel_initializer, conv_flag=True, bnflag=True,  relu=True)
      
      self.conv_bn_3x3 = ConvBnRl(filters=self.filters, kernel_size=(3,3), strides=(1,1), padding="same" , dilation_rate=(1,1), 
                                  kernel_regularizer = self.kernel_regularizer, kernel_initializer=self.kernel_initializer, conv_flag=True, bnflag=True,  relu=True)
      
      
      self.cardinality = cardinality
      
      self.pool = pool
      
    def concatenation(self, layers) :
      
            return tf.keras.layers.concatenate(layers, axis=3)       

    def first_layer(self, x, scope):
        with tf.name_scope(scope) :
            
            x = self.conv_bn_3x3(x)

            return x

    def transform_layer(self, x, depth, pool_flag, scope):
        with tf.name_scope(scope) :
          
            if pool_flag:
              
              x = self.pool(x)
          
            cbr_1x1 = ConvBnRl(filters=depth, kernel_size=(1,1), strides=(1,1), padding="same" , dilation_rate=(1,1), 
                                  kernel_regularizer = self.kernel_regularizer, kernel_initializer=self.kernel_initializer, conv_flag=True, bnflag=True,  relu=True)
            
            x = cbr_1x1(x)
            
            cbr_3x3 = ConvBnRl(filters=depth, kernel_size=(3,3), strides=(1,1), padding="same" , dilation_rate=(1,1), 
                                  kernel_regularizer = self.kernel_regularizer, kernel_initializer=self.kernel_initializer, conv_flag=True, bnflag=True,  relu=True)
            
            x = cbr_3x3(x)

            return x

    def transition_layer(self, x, filters, scope):
        with tf.name_scope(scope):
          
            cb_1x1 = ConvBnRl(filters=filters, kernel_size=(1,1), strides=(1,1), padding="same" , dilation_rate=(1,1), 
                                  kernel_regularizer = self.kernel_regularizer, kernel_initializer=self.kernel_initializer, conv_flag=True, bnflag=True,  relu=True)
            
            x = cb_1x1(x)

            return x

    def split_layer(self, input_x, filters, pool_flag, layer_name):
      
        with tf.name_scope(layer_name) :
          
            layers_split = list()
            
            depth = filters//self.cardinality
            
            for i in range(self.cardinality) :
              
                splits = self.transform_layer(input_x, depth, pool_flag=pool_flag, scope=layer_name + '_splitN_' + str(i))
                
                layers_split.append(splits)

            return self.concatenation(layers_split)

    def residual_layer(self, input_x):
        # split + transform(bottleneck) + transition + merge

        for i in range(self.res_block):
            # input_dim = input_x.get_shape().as_list()[-1]
            input_dim = int(np.shape(input_x)[-1])

            if input_dim * 2 == self.filters:
              
                flag = True
                
                stride = 2
                
                channel = input_dim // 2
                
            else:
              
                flag = False
                
                stride = 1
                
            x = self.split_layer(input_x, filters=self.filters, pool_flag=flag, layer_name='split_layer_'+self.layer_num+'_'+str(i))
            
            x = self.transition_layer(x, filters=self.filters, scope='trans_layer_'+self.layer_num+'_'+str(i))

            if flag is True :
                
                pad_input_x = self.pool(input_x)
                
                pad_input_x = tf.pad(pad_input_x, [[0, 0], [0, 0], [0, 0], [channel, channel]]) # [?, height, width, channel]
                
                
                
            else :
                pad_input_x = input_x
                
            input_x = tf.nn.relu(x + pad_input_x)
        
        return input_x
      
    def call(self, inputs_x):
        
        return self.residual_layer(inputs_x)
      

In [12]:
b = ResNeXtBlk(layer_num="1")

b(a(x(np.random.normal(size=(32,32,3)).reshape(1,32,32,3).astype(np.float16))))

<tf.Tensor 'res_ne_xt_blk_1/Relu:0' shape=(1, 16, 16, 32) dtype=float16>

# Inception

In [11]:
"""class InceptionBlk(tf.keras.Model):
  """

<tf.Tensor 'res_ne_xt_blk/Relu:0' shape=(1, 16, 16, 32) dtype=float16>

# ResNext_50

In [0]:
class ResNext_50(tf.keras.Model):
  
  def __init__(self, num_classes= 10, f_filter=32):
    
    super().__init__()

    
    self.first_layer = ConvBnRl(filters=f_filter, kernel_size=(1,1), strides=(1,1), padding="same" , dilation_rate=(1,1), 
                                  kernel_regularizer = None, kernel_initializer='glorot_uniform', conv_flag=True, bnflag=True,  relu=True)

    self.blk1 = ResNeXtBlk(filters=f_filter, layer_num='1')

    self.blk2 = ResNeXtBlk(filters=f_filter*2, layer_num='2')

    self.blk3 = ResNeXtBlk(filters=f_filter*4, layer_num='3')

    self.pool = tf.keras.layers.GlobalMaxPool2D()

    self.linear = tf.keras.layers.Dense(num_classes, kernel_initializer='glorot_uniform', use_bias=False)
  
  def call(self, x, y):
        
        x = self.first_layer(x)

        x = self.blk1(x)
        
        x = self.blk2(x)
        
        x = self.blk3(x)

        x = self.pool(x)
        
        x = self.linear(x)
        
        ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=x, labels=y)
    
        loss = tf.reduce_sum(ce)
    
        correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(x, axis = 1), y), tf.float32))
      
        return loss, correct
    

In [32]:
resnext_model = ResNext_50(num_classes=10)

resnext_model(np.random.normal(size=(32,32,3)).reshape(1,32,32,3).astype(np.float16), 
              np.array([1]))

(<tf.Tensor 'res_next_50_6/Sum:0' shape=() dtype=float16>,
 <tf.Tensor 'res_next_50_6/Sum_1:0' shape=() dtype=float32>)

# DavidNet

In [0]:
class DavidNet(tf.keras.Model):
  
  def __init__(self, num_classes= 10, f_filter=32, weight=0.125):
    
    super().__init__()
    
    self.init_conv_bn = ConvBnRl(filters=f_filter, kernel_size=(1,1), strides=(1,1), padding="same" , dilation_rate=(1,1), 
                                  kernel_regularizer = None, kernel_initializer='glorot_uniform', conv_flag=True, bnflag=True,  relu=True)
    
    self.blk1 = ResBlk(cbr = ConvBnRl(filters=f_filter, kernel_size=(3,3), strides=(1,1), padding="same" , conv_flag=True, bnflag=True, relu=True),
               
               cbr_res1 = ConvBnRl(filters=f_filter, kernel_size=(3,3), strides=(1,1), padding="same" , conv_flag=True, bnflag=True, relu=True),
               
               cbr_res2 = ConvBnRl(filters=f_filter, kernel_size=(3,3), strides=(1,1), padding="same" , conv_flag=True, bnflag=True, relu=True),
               
               pool=tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=None, padding='same'), 
               
               res=True )
    
    self.blk2 = ResBlk(cbr = ConvBnRl(filters=f_filter*2, kernel_size=(3,3), strides=(1,1), padding="same" , conv_flag=True, bnflag=True, relu=True),
               
               cbr_res1 = ConvBnRl(filters=f_filter*2, kernel_size=(3,3), strides=(1,1), padding="same" , conv_flag=True, bnflag=True, relu=True),
               
               cbr_res2 = ConvBnRl(filters=f_filter*2, kernel_size=(3,3), strides=(1,1), padding="same" , conv_flag=True, bnflag=True, relu=True),
               
               pool=tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=None, padding='same'), 
               
               res=True )
    
    self.blk3 = ResBlk(cbr = ConvBnRl(filters=f_filter*3, kernel_size=(3,3), strides=(1,1), padding="same" , conv_flag=True, bnflag=True, relu=True),
               
               cbr_res1 = ConvBnRl(filters=f_filter*3, kernel_size=(3,3), strides=(1,1), padding="same" , conv_flag=True, bnflag=True, relu=True),
               
               cbr_res2 = ConvBnRl(filters=f_filter*3, kernel_size=(3,3), strides=(1,1), padding="same" , conv_flag=True, bnflag=True, relu=True),
               
               pool=tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=None, padding='same'), 
               
               res=True )
    
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    
    self.linear = tf.keras.layers.Dense(num_classes, kernel_initializer='glorot_uniform', use_bias=False)
    
    self.weight = weight

  def call(self, x, y):
    
    h = self.pool(self.blk3(self.blk2(self.blk1(self.init_conv_bn(x)))))
    
    h = self.linear(h) * self.weight
    
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    
    loss = tf.reduce_sum(ce)
    
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float16))
    
    return loss, correct

In [35]:
davidnet_model = DavidNet(num_classes=10)

davidnet_model(np.random.normal(size=(32,32,3)).reshape(1,32,32,3).astype(np.float16), 
              np.array([1]))

(<tf.Tensor 'david_net/Sum:0' shape=() dtype=float16>,
 <tf.Tensor 'david_net/Sum_1:0' shape=() dtype=float16>)